#### Nota:
Es de importancia destacar que no se utilizarán todos los campos de relevancia, pero que bien podrían ser utilizados por fuera del contexto de este MVP.
Campos como la colección a la que pertenecen, las compañías que producen las filmaciones y los países en los que se producen pueden ser igualmente categorizados mediante el mismo proceso de los lenguajes y generos, interrelacionando tanto colecciones y compañías con géneros particulares de forma en que obtengamos un índice numérico que nos permita caracterizar a la compañía, para dicho caso, en un vector de coordenadas sobre el cual podremos obtener aquellas compañías que produzcan contenido similar y, por tanto, tengan coordenadas cercanas.

En el caso de los lenguajes pueden ser bien usados como un filtro que nos permita considerar exclusivamente aquellas películas que compartan lenguaje con la película elegida, mientras que un análisis, similar al hecho sobre los lenguajes, puede ser realizado sobre los países en los que se producen las películas, de forma que podamos recomendar aquellas películas que también compartan dichos países de manera prioritaria.

**Ninguna de estas posibilidades va a ser considerada para la formación del modelo de recomendación visto en este MVP**

El objetivo de considerar solo aquellos lenguajes que se compartan entre las películas es no solo recomendar contenido únicamente en los lenguajes hablados por la persona que ve la película sino también cumple la función de reducir el tamaño sobre el cual se efectúan las operaciones del modelo.

Otro factor a considerar es a través de las descripciones de cada película, cosa que no ha sido considerada en el analisis EDA pero que a través de un análisis de palabras claves o de similitud con el resto de descripciones en las películas a considerar por el modelo, nos permitiría extraer contenido más relevante, como serían películas de la misma trama pero que no necesariamente comparten etiquetas o fueren igual de exitosas en popularidad.

La librería de machine learning utilizada en este modelo es **sklearn**, que incluye también funciones y modelos para este propósito, más para el propósito actual utilizo la función de similitud del coseno, que nos permite encontrar aquel vector de coordenadas (el entre los films del set) qué más se aproxime al que representa nuestra película, considerando las features que hemos tomado anteriormente como coordenadas.

A través de este análisis se podrá determinar la cercanía de cada una de las películas, que el modelo analiza, con nuestra pelicula inicial. Es tomando aquellas que tengan un mayor resultados según este modelo qué obtendríamos las películas a recomendar

In [52]:
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [53]:
data = pd.read_csv('datasets/ml_encoded_data.csv')
data.shape

(13051, 40)

In [54]:
scaler = StandardScaler()
features = data[['popularity', 'return', 'vote_average', 'release_year', 'runtime']]
scaled_features = pd.DataFrame(scaler.fit_transform(features.values))
scaled_features.shape

(13051, 5)

In [55]:
cols =data[["id",'16','35','10751','12','14','10749','18','28','80','53','27','36','878','9648','10752','10402','99','37','10769','10770','en','fr','es','de','it','ja','ru','zh','pt','cn','other_lang']]
cols.shape

(13051, 32)

In [ ]:
final_features= pd.concat([scaled_features, cols], axis=1)
final_features.head(1)


,0,1,2,3,4,id,16,35,10751,12,...,fr,es,de,it,ja,ru,zh,pt,cn,other_lang
0,-0.451793,-0.015187,0.929453,-0.556816,-1.211234,2,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [57]:
final_features.fillna(0, inplace=True)

In [58]:
movie_df = pd.read_csv("datasets/movies_final.csv", index_col = ["id"])
movie_df.head()

,title,release_date,belongs_to_collection,budget,revenue,genres,original_language,overview,popularity,production_companies,production_countries,runtime,spoken_languages,status,tagline,vote_average,vote_count,return,release_year
id,,,,,,,,,,,,,,,,,,,
862,Toy Story,1995-10-30,10194.0,30000000,373554033.0,"[16, 35, 10751]",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[3],['US'],81.0,['en'],Released,NaN,7.7,5415,12.451801,1995
8844,Jumanji,1995-12-15,NaN,65000000,262797249.0,"[12, 14, 10751]",en,When siblings Judy and Peter discover an encha...,17.015539,"[559, 2550, 10201]",['US'],104.0,"['en', 'fr']",Released,Roll the dice and unleash the excitement!,6.9,2413,4.043035,1995
15602,Grumpier Old Men,1995-12-22,119050.0,0,0.0,"[10749, 35]",en,A family wedding reignites the ancient feud be...,11.712900,"[6194, 19464]",['US'],101.0,['en'],Released,Still Yelling. Still Fighting. Still Ready for...,6.5,92,0.000000,1995
31357,Waiting to Exhale,1995-12-22,NaN,16000000,81452156.0,"[35, 18, 10749]",en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[306],['US'],127.0,['en'],Released,Friends are the people who let you be yourself...,6.1,34,5.090760,1995
11862,Father of the Bride Part II,1995-02-10,96871.0,0,76578911.0,[35],en,Just when George Banks has recovered from his ...,8.387519,"[5842, 9195]",['US'],106.0,['en'],Released,Just When His World Is Back To Normal... He's ...,5.7,173,0.000000,1995


In [114]:
def recomendacion(titulo, num_recommendations=5):

    try:
        movie_data= movie_df[movie_df['title'].str.lower() == titulo.lower()].iloc[0]
    except IndexError:
        return f"Movie '{titulo}' not found in the dataset."
    
    movie_features = final_features[final_features["id"]==movie_data.name].drop(columns="id").values.reshape(1, -1)
    # Compute cosine similarity with the feature matrix
    
    similaryties = list(enumerate(cosine_similarity(movie_features,final_features.drop(columns="id"))[0]))
    
    sim = sorted(similaryties, reverse=True, key=lambda t: t[1])
    # Get indices of the most similar movies (skip the first one because it's the same movie)
    
    similar_indices =sim[1:num_recommendations + 1]
    idx = [final_features.loc[i[0], "id"] for i in similar_indices ]
    # Return recommended movie titles
    return movie_df.loc[idx]["title"].tolist()

In [115]:
recomendacion("Toy Story")

['Toy Story 2',
 'Monsters, Inc.',
 'Lilo & Stitch',
 'The Simpsons Movie',
 'Finding Nemo']

##### Coclusion
En este estado las recomendaciones son bastante relevantes, pero tambien pueden mejorarse al aumentar la complejidad del modelo